#### TELECOM CHURN MANAGEMENT - SEGMENTATION - CLASIFICATION ( DATA CLEANING - ACTIVE CUSTOMERS)

    #1. Understand the data and perform the data analysis on active customers' data

    A. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import scipy.stats as stats
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import pandas_profiling
%matplotlib inline

C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas_profiling\base.py:20: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\runpy.py", line 174, in _run_module_as_main
    "__main__

    B. Importing data 

In [2]:
telecom_active = pd.read_csv("telecom_active.csv")

    C. PURIFYING DATA FOR FURTHER ANALYSIS

In [3]:
telecom_active.head()

,SITE_ACCOUNT_NUMBER,ACCOUNT_STATUS,FIRST_COMMUNICATION_DATE,UPGRADE_DATE,SERVICE_ PLAN,MODEL,SALES_CHANNEL,DIRECT_INDIRECT_CHANNEL,SALES_SOURCE,COMPANY_SOURCE_NAME,...,SITE_TYPE_DESC,WARRANTY_NAME,MOST_RECENT_SALES_CHANNEL,GENDER,DATE_OF_BIRTH,AGE,INCOME,MARITAL_STATUS,PRESENCE_OF_CHILDREN,COMPUTER_OWNER
0,AMR-11112345,Active,27-11-2001,14-12-2003,NaN,HN9000,Vars,Indirect,UNKNOWN,HNS Customers,...,Broadband Remote,NaN,HN7000 Upgrade,2.0,01-05-1943,69,0,1,0,N
1,AMR-14837287,Active,02-05-2003,08-08-2006,Pro,HN7000S,Sales Agents,Indirect,NCC,HNS Customers,...,Spaceway,NaN,Retail,2.0,28-12-1969,43,4,1,0,N
2,AMR-14837803,Active,28-01-2001,17-03-2004,Power150,HN9000,Call Center,Indirect,NCC,HNS Customers,...,Broadband Remote,NaN,National Sales,2.0,20-07-1954,58,4,1,0,N
3,AMR-14837821,Active,20-11-2001,27-03-2005,Power150,DW6000,Sales Agents,Direct,NaN,HNS Customers,...,Spaceway,NaN,Direct,NaN,10-05-1990,22,6,1,1,NaN
4,AMR-14839297,Active,27-02-2002,14-06-2004,Home,HN7000S,Call Center,Indirect,TAG,HNS Customers,...,Broadband Remote,NaN,DW6000 Upgrade,1.0,11-05-1976,36,7,0,0,NaN


In [4]:
pandas_profiling.ProfileReport(telecom_active)

Number of variables,29
Number of observations,9998
Total Missing (%),8.4%
Total size in memory,2.2 MiB
Average record size in memory,232.0 B
Numeric,2
Categorical,25
Date,0
Text (Unique),1
Rejected,1
Distinct count,2


    Dropping variables with highest cardinality and non retrospective - 

In [5]:
telecom_active.drop(['ACCOUNT_STATUS' , 'ADDRESS', 'CITY', 'COMPANY_SOURCE_NAME', 'DATE_OF_BIRTH', 'EMAIL_ADDRESS', 'FIRST_COMMUNICATION_DATE', 'FIRST_NAME', 'LAST_NAME', 'ZIPCODE','UPGRADE_DATE','STATE','SALES_SOURCE'], axis=1, inplace = True)

In [6]:
telecom_active.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 16 columns):
SITE_ACCOUNT_NUMBER          9998 non-null object
SERVICE_ PLAN                8760 non-null object
MODEL                        9332 non-null object
SALES_CHANNEL                7093 non-null object
DIRECT_INDIRECT_CHANNEL      9955 non-null object
FEEDBACK                     8084 non-null object
SATELLITE                    8772 non-null object
SITE_TYPE_DESC               9998 non-null object
WARRANTY_NAME                535 non-null object
MOST_RECENT_SALES_CHANNEL    9984 non-null object
GENDER                       8727 non-null float64
AGE                          9998 non-null int64
INCOME                       9694 non-null object
MARITAL_STATUS               9691 non-null object
PRESENCE_OF_CHILDREN         8695 non-null object
COMPUTER_OWNER               6944 non-null object
dtypes: float64(1), int64(1), object(14)
memory usage: 1.2+ MB


In [7]:
telecom_active.isnull().any()

SITE_ACCOUNT_NUMBER          False
SERVICE_ PLAN                 True
MODEL                         True
SALES_CHANNEL                 True
DIRECT_INDIRECT_CHANNEL       True
FEEDBACK                      True
SATELLITE                     True
SITE_TYPE_DESC               False
WARRANTY_NAME                 True
MOST_RECENT_SALES_CHANNEL     True
GENDER                        True
AGE                          False
INCOME                        True
MARITAL_STATUS                True
PRESENCE_OF_CHILDREN          True
COMPUTER_OWNER                True
dtype: bool

    Replacing #REF and NULL of Categorical variables as 'NA'

In [8]:
telecom_active[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL' ]] = telecom_active[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL' ]].replace(np.nan, 'NA', regex=True)

In [9]:
telecom_active[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL' ]] = telecom_active[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL' ]].replace('REF!', 'NA', regex=True)

    Replacing #REF and NULL of Numeric Varaibles as '9' and converting the variable type to int64
    9 - indicates unknown

In [10]:
telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]] = telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]].replace('#REF!', '9', regex=True)

In [11]:
telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]] = telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]].replace(np.nan, '9', regex=True)

    For AGE = 0 mask them with mean age of population

In [12]:
MEAN_AGE_ACTIVE = telecom_active['AGE'].mean()
telecom_active['AGE']=telecom_active.AGE.mask(telecom_active.AGE == 0,MEAN_AGE_ACTIVE)

In [13]:
telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN','AGE' ]] = telecom_active[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN','AGE']].astype('int64')

In [14]:
telecom_active.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 16 columns):
SITE_ACCOUNT_NUMBER          9998 non-null object
SERVICE_ PLAN                8760 non-null object
MODEL                        9332 non-null object
SALES_CHANNEL                9998 non-null object
DIRECT_INDIRECT_CHANNEL      9998 non-null object
FEEDBACK                     9998 non-null object
SATELLITE                    8772 non-null object
SITE_TYPE_DESC               9998 non-null object
WARRANTY_NAME                535 non-null object
MOST_RECENT_SALES_CHANNEL    9984 non-null object
GENDER                       9998 non-null int64
AGE                          9998 non-null int64
INCOME                       9694 non-null object
MARITAL_STATUS               9998 non-null int64
PRESENCE_OF_CHILDREN         9998 non-null int64
COMPUTER_OWNER               9998 non-null object
dtypes: int64(4), object(12)
memory usage: 1.2+ MB


In [15]:
telecom_active.isnull().any()

SITE_ACCOUNT_NUMBER          False
SERVICE_ PLAN                 True
MODEL                         True
SALES_CHANNEL                False
DIRECT_INDIRECT_CHANNEL      False
FEEDBACK                     False
SATELLITE                     True
SITE_TYPE_DESC               False
WARRANTY_NAME                 True
MOST_RECENT_SALES_CHANNEL     True
GENDER                       False
AGE                          False
INCOME                        True
MARITAL_STATUS               False
PRESENCE_OF_CHILDREN         False
COMPUTER_OWNER               False
dtype: bool

    Treating Catergorical variables with moderate cardinality

    STEP 01 - Mising value treatment

In [16]:
telecom_active[['SERVICE_ PLAN', 'MODEL', 'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL' ]] = telecom_active[['SERVICE_ PLAN', 'MODEL', 'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL']].replace('#REF!', 'NA', regex=True)

In [17]:
telecom_active[['SERVICE_ PLAN', 'MODEL', 'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL' ]] = telecom_active[['SERVICE_ PLAN', 'MODEL', 'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL']].replace(np.nan, 'NA', regex=True)

    STEP 02 - Working with column names and column values
              Special consideration on unwanted spaces and squeezing vvariables wherever required

###### COLUMN NAME - SERVICE_PLAN  

In [18]:
telecom_active.groupby('SERVICE_ PLAN')['SITE_ACCOUNT_NUMBER'].nunique()

SERVICE_ PLAN
Basic        1263
Basic_Ltd    1268
Elite        1256
Home         1243
NA           1238
Power150     1248
Pro          1210
Proplus      1272
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [19]:
telecom_active['SERVICE_PLAN_CAT'] = telecom_active['SERVICE_ PLAN' ].str.strip()

In [20]:
telecom_active.drop('SERVICE_ PLAN', axis =1 , inplace = True)

###### COLUMN NAME - MODEL

In [21]:
telecom_active.groupby('MODEL')['SITE_ACCOUNT_NUMBER'].nunique()

MODEL
DW4000      645
DW4020      655
DW6000      514
DW7000      251
HN7000S    4296
HN9000     1985
NA          675
UNK         977
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [22]:
telecom_active['MODEL_CAT'] = telecom_active['MODEL' ].str.strip()

In [23]:
telecom_active.drop('MODEL', axis = 1, inplace = True)

##### COLUMN NAME - MOST_RECENT_SALES_CHANNEL
    - Creating new categorical variable MOST_RECENT_SALES_CHANNEL_CAT
    - Replacing "DW6000 Upgrade" with "DW6K Upgrade", "DW7000 Upgrade" with "DW7K Upgrade"
    - Replacing "HN7000 Upgrade", "HN9000 Upgrade" with "HN9K Upgrade"

In [24]:
telecom_active.groupby('MOST_RECENT_SALES_CHANNEL')['SITE_ACCOUNT_NUMBER'].nunique()

MOST_RECENT_SALES_CHANNEL
Business Edition            6
Classic Migration         259
DPC Classic upgrade       995
DW6000 Upgrade           1076
DW7000 Upgrade            498
Dealer-Generated Sale    1292
Dial Return Upgrade       214
Direct                   1499
HN7000 Upgrade           1709
HN9000 Upgrade            707
NA                         19
National Sales            368
Retail                    271
Retail                    995
UNKNOWN                    76
Used Equipment             14
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [25]:
telecom_active['MOST_RECENT_SALES_CHANNEL_CAT'] = telecom_active['MOST_RECENT_SALES_CHANNEL'].str.strip()

In [26]:
telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(["DW6000 Upgrade"], "DW6K Upgrade", regex=True)

In [27]:
telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['DW7000 Upgrade'], 'DW7K Upgrade', regex=True)

In [28]:
telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['HN7000 Upgrade'], 'HN7K Upgrade', regex=True)

In [29]:
telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['HN9000 Upgrade'], 'HN9K Upgrade', regex=True)

In [30]:
telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_active[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['0','UNKNOWN'], 'NA', regex=True)

In [31]:
telecom_active.groupby('MOST_RECENT_SALES_CHANNEL_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

MOST_RECENT_SALES_CHANNEL_CAT
Business Edition            6
Classic Migration         259
DPC Classic upgrade       995
DW6K Upgrade             1076
DW7K Upgrade              498
Dealer-Generated Sale    1292
Dial Return Upgrade       214
Direct                   1499
HN7K Upgrade             1709
HN9K Upgrade              707
NA                         95
National Sales            368
Retail                   1266
Used Equipment             14
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [32]:
telecom_active.drop('MOST_RECENT_SALES_CHANNEL', axis =1, inplace = True)

##### COLUMN NAME - SATELLITE
    There are lots of cardinality in the column - so bringing the same types together

In [33]:
telecom_active.groupby('SATELLITE')['SITE_ACCOUNT_NUMBER'].nunique()

SATELLITE
AM32-HOR-19K      54
AM32-HOR-7K       26
AM6-HOR-21K      667
AM9-HOR-17K       34
AM9-HOR-19K       23
AM9-HOR-21K       61
AM9-VER-22K       19
AM9-VER-2K         9
AM9-VER-2K       358
AMC-HOR-10K       13
AMC-VER-5K         8
G16-HOR-2K        31
G16-HOR-8K        75
G16-VER-15K      146
G16-VER-1K        78
G16-VER-21K       44
G16-VER-2K        23
G17-HOR-2K       649
G17-HOR-6K       720
G26-HOR-14K       16
G28-HOR-10K       12
G28-HOR-16K       18
G3C-VER-21K       99
G3C-VER-24K       67
G3C-VER-9K        42
HZN-HOR-19K       49
HZN-HOR-8K        51
HZN-VER-11K       23
HZN-VER-23K       50
HZN-VER-6K        32
HZN2-VER-4K      254
HZN2-VER-5K       32
HZN2-VER-8K       67
NA              1237
SM5-HOR-14K      346
SM5-HOR-6K       429
SM5-VER-15K      814
SM5-VER-1K       564
SM5-VER-5K       887
SM5-VER-7K       390
SM6-HOR-10K       12
SM6-HOR-12K       27
SM6-HOR-13K       29
SM6-VER-2K        26
SPACEWAY3       1387
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [34]:
telecom_active['SATELLITE_CAT'] = telecom_active['SATELLITE'].str.strip()

In [35]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["AM32-HOR-19K","AM32-HOR-7K","AM6-HOR-21K","AM9-HOR-17K","AM9-HOR-19K","AM9-HOR-21K","AMC-HOR-10K"], "AM-HOR-SERIES", regex=True)

In [36]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["AM9-VER-22K","AM9-VER-2K","AM9-VER-2K","AMC-VER-5K"], "AM-VER-SERIES", regex=True)

In [37]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["G16-HOR-2K","G16-HOR-8K","G16-VER-15K","G17-HOR-2K","G17-HOR-6K","G26-HOR-14K","G28-HOR-10K","G28-HOR-16K"], "G-HOR-SERIES", regex=True)

In [38]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["G16-VER-15K","G16-VER-1K","G16-VER-21K","G16-VER-2K","G3C-VER-21K","G3C-VER-24K","G3C-VER-9K"], "G-VER-SERIES", regex=True)

In [39]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["HZN-HOR-19K","HZN-HOR-8K"], "HZN-HOR-SERIES", regex=True)

In [40]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["HZN2-VER-4K","HZN2-VER-5K","HZN2-VER-8K","HZN-VER-11K","HZN-VER-23K","HZN-VER-6K"], "HZN-VER-SERIES", regex=True)

In [41]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["SM5-HOR-14K", "SM5-HOR-6K"], "SM5-HOR-SERIES", regex=True)

In [42]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["SM5-VER-15K", "SM5-VER-1K", "SM5-VER-5K","SM5-VER-7K"], "SM5-VER-SERIES", regex=True)

In [43]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["SM6-HOR-10K", "SM6-HOR-12K", "SM6-HOR-13K"], "SM6-HOR-SERIES", regex=True)

In [44]:
telecom_active[['SATELLITE_CAT']] = telecom_active[['SATELLITE_CAT']].replace(["SM6-VER-2K"], "SM6-VER-SERIES", regex=True)

In [45]:
telecom_active.groupby('SATELLITE_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

SATELLITE_CAT
AM-HOR-SERIES      878
AM-VER-SERIES      394
G-HOR-SERIES      1667
G-VER-SERIES       353
HZN-HOR-SERIES     100
HZN-VER-SERIES     458
NA                1237
SM5-HOR-SERIES     775
SM5-VER-SERIES    2655
SM6-HOR-SERIES      68
SM6-VER-SERIES      26
SPACEWAY3         1387
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [46]:
telecom_active.drop('SATELLITE', axis =1 , inplace = True)

##### COLUMN NAME - WARRANTY_NAME

In [47]:
telecom_active.groupby('WARRANTY_NAME')['SITE_ACCOUNT_NUMBER'].nunique()

WARRANTY_NAME
EXPRESS REPAIR BASIC                               50
EXPRESS REPAIR BUSINESS WITH TERM                  38
EXPRESS REPAIR HOME                                44
EXPRESS REPAIR HOME W/O COMMITMENT WITH TERM       20
EXPRESS REPAIR HOME WITH COMMITMENT WITH TERM      52
EXPRESS REPAIR HOME WITH TERM                     295
NA                                               9475
SRVC ASSUR STD EXISTING CUST NO COMMIT             17
SRVC ASSUR STD NEW CUST                             7
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [48]:
telecom_active['WARRANTY_NAME_CAT'] = telecom_active['WARRANTY_NAME'].str.strip()

In [49]:
telecom_active.groupby('WARRANTY_NAME_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

WARRANTY_NAME_CAT
EXPRESS REPAIR BASIC                               50
EXPRESS REPAIR BUSINESS WITH TERM                  38
EXPRESS REPAIR HOME                                44
EXPRESS REPAIR HOME W/O COMMITMENT WITH TERM       20
EXPRESS REPAIR HOME WITH COMMITMENT WITH TERM      52
EXPRESS REPAIR HOME WITH TERM                     295
NA                                               9475
SRVC ASSUR STD EXISTING CUST NO COMMIT             17
SRVC ASSUR STD NEW CUST                             7
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [50]:
telecom_active.drop('WARRANTY_NAME', axis =1, inplace = True)

    Treating #REF! value of INCOME with median (max repeatitive value) for active customers

In [51]:
# 0 - is the max repleatative value for active customers
telecom_active.groupby('INCOME')['SITE_ACCOUNT_NUMBER'].nunique()

INCOME
#REF!      13
0        2734
1         156
2         169
3         669
4         963
5         876
6        1545
7        1015
8         597
9         464
A         193
B          43
C          99
D         158
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [52]:
telecom_active[['INCOME' ]] = telecom_active[['INCOME']].replace('#REF!', '0', regex=True)

In [53]:
telecom_active[['INCOME' ]] = telecom_active[['INCOME']].replace(np.nan, '0', regex=True)

    D. Creating new KPIs for active customers' dataset
    - CHURN_IND 0 implying NOT churned, 1 implying churned
    - UPGRADE_IND 0 implying NOT upgraded, 1 implying upgraded
      (For all active customers UPGRADE_DATE is availible )

In [54]:
telecom_active['CHURN_IND'] = 0

In [55]:
telecom_active['UPGRADE_IND'] = 1

In [56]:
telecom_active.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 18 columns):
SITE_ACCOUNT_NUMBER              9998 non-null object
SALES_CHANNEL                    9998 non-null object
DIRECT_INDIRECT_CHANNEL          9998 non-null object
FEEDBACK                         9998 non-null object
SITE_TYPE_DESC                   9998 non-null object
GENDER                           9998 non-null int64
AGE                              9998 non-null int64
INCOME                           9998 non-null object
MARITAL_STATUS                   9998 non-null int64
PRESENCE_OF_CHILDREN             9998 non-null int64
COMPUTER_OWNER                   9998 non-null object
SERVICE_PLAN_CAT                 9998 non-null object
MODEL_CAT                        9998 non-null object
MOST_RECENT_SALES_CHANNEL_CAT    9998 non-null object
SATELLITE_CAT                    9998 non-null object
WARRANTY_NAME_CAT                9998 non-null object
CHURN_IND                        

In [57]:
telecom_active.isnull().any()

SITE_ACCOUNT_NUMBER              False
SALES_CHANNEL                    False
DIRECT_INDIRECT_CHANNEL          False
FEEDBACK                         False
SITE_TYPE_DESC                   False
GENDER                           False
AGE                              False
INCOME                           False
MARITAL_STATUS                   False
PRESENCE_OF_CHILDREN             False
COMPUTER_OWNER                   False
SERVICE_PLAN_CAT                 False
MODEL_CAT                        False
MOST_RECENT_SALES_CHANNEL_CAT    False
SATELLITE_CAT                    False
WARRANTY_NAME_CAT                False
CHURN_IND                        False
UPGRADE_IND                      False
dtype: bool

In [58]:
telecom_active.to_csv("telecom_active_cleaned.csv", index= False)

In [59]:
pandas_profiling.ProfileReport(telecom_active)

Number of variables,18
Number of observations,9998
Total Missing (%),0.0%
Total size in memory,1.4 MiB
Average record size in memory,144.0 B
Numeric,4
Categorical,11
Date,0
Text (Unique),1
Rejected,2
Distinct count,62


    ************************************ END OF PART 1 ****************************************************